# **Libs**

In [ ]:
# Torch
from torch.utils.data import DataLoader
from torchvision import transforms, models

# Остальное
from sklearn.model_selection import train_test_split

# Utils
from image_classification.model import *

# **Code**

## Зафиксируем seed

In [ ]:
set_all_seeds()

## Data

### **Transformation** and **augmentation**

In [ ]:
# Определим базовые преобразования
image_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),                      # Преобразовать в тензор
    transforms.Normalize(mean=mean, std=std)    # Нормализовать данные
])

# Определим преобразования с аугментациями
image_augmentation = transforms.Compose([
    image_transform
])

### Reading

In [ ]:
classes = list()

In [ ]:
image_paths = list()
labels = list()

### Split

In [ ]:
train_image_paths, valid_image_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42, stratify=labels)

### Create **Datasets**

In [ ]:
dataset = ImageClassificationDataset(image_paths, labels, transform=image_transform)

train_set = ImageClassificationDataset(train_image_paths, train_labels, transform=image_augmentation)
valid_set = ImageClassificationDataset(valid_image_paths, valid_labels, transform=image_transform)

### Visualization

In [ ]:
show_images(dataset, classes=classes)

### Creating a DataLoader

In [ ]:
# Создание DataLoader для каждой выборки
batch_size = 24

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

## Models

### Score

In [ ]:
scores = dict()

### EfficientNet

In [ ]:
efficientnet_raw = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
efficientnet_raw.classifier[-1] = nn.Linear(efficientnet_raw.classifier[-1].in_features, len(classes))

In [ ]:
efficientnet = ImageClassifier(efficientnet_raw, "EfficientNet")

In [ ]:
efficientnet.fit(train_loader, valid_loader, 10)
scores[efficientnet.best_score] = efficientnet

## Result

In [ ]:
best_model = scores[max(scores)] # ? Выбрать модель
best_model.name

In [ ]:
# Количество изображений для отображения
n = 3

# Создание фигуры для вывода нескольких изображений
fig, axes = plt.subplots(n, 1, figsize=(5, 5 * n))
fig.patch.set_alpha(0.0)  # Прозрачный фон

for i, idx in enumerate(random.sample(range(len(valid_set)), n)):
    # Извлечение данных и предсказания
    image, label = valid_set[idx]  # Предполагается, что valid_set возвращает (изображение, метка)
    prediction = best_model.predict(image)  # Предсказание для одного изображения

    # Денормализация изображения и отображение
    ax = axes[i]
    ax.imshow(denormalize(image).cpu().numpy().transpose(1, 2, 0))
    ax.axis('off')  # Отключаем оси
    ax.set_title(f"Class: {classes[label]}\nPredict: {classes[prediction]}", fontsize=10, color='white')

# Общий показ фигуры
plt.tight_layout()
plt.show()

## Submission

In [ ]:
test_image_names = list(map(lambda path: f"../data/test_images/{path}", os.listdir("../data/test_images")))
test_set = ImageDataset(test_image_names, transform=image_transform)

In [ ]:
predict_class_id = best_model.predict(test_set)
predict_class_names = [classes[class_id] for class_id in predict_class_id]